In [4]:
import openai
import os
import json
import pandas as pd
from pykrx import stock
from datetime import datetime
from difflib import get_close_matches

# 🔑 OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 주가 분석 함수 (오타 시 유사 종목 추천 포함)
def analyze_stock_price(stock_name, start_date, end_date):
    # 전체 종목 코드/이름 목록
    tickers = stock.get_market_ticker_list()
    name_to_code = {stock.get_market_ticker_name(ticker): ticker for ticker in tickers}
    stock_names = list(name_to_code.keys())

    if stock_name not in name_to_code:
        # 유사한 종목명 추천
        suggestions = get_close_matches(stock_name, stock_names, n=3, cutoff=0.6)
        if suggestions:
            suggest_msg = "\n💡 입력하신 종목명과 유사한 종목:\n- " + "\n- ".join(suggestions)
            return f"❌ '{stock_name}'은(는) 유효한 종목명이 아닙니다.{suggest_msg}"
        else:
            return f"❌ '{stock_name}'은(는) 유효한 종목명이 아닙니다. 다시 확인해 주세요."

    code = name_to_code[stock_name]
    
    # 날짜 변환
    start_fmt = pd.to_datetime(start_date).strftime("%Y%m%d")
    end_fmt = pd.to_datetime(end_date).strftime("%Y%m%d")
    
    # 주가 데이터 수집
    df = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, code)
    if df.empty:
        return f"📭 {stock_name}({code})의 주가 데이터가 {start_date} ~ {end_date} 기간에 없습니다."

    avg_price = df['종가'].mean()
    max_price = df['종가'].max()
    min_price = df['종가'].min()

    summary = f"""
📈 {stock_name} ({code}) 주가 요약 ({start_date} ~ {end_date}):
- 평균 종가: {avg_price:,.0f}원
- 최고 종가: {max_price:,.0f}원
- 최저 종가: {min_price:,.0f}원
"""
    return summary

# 🔧 GPT Function 설정
functions = [
    {
        "name": "analyze_stock_price",
        "description": "한국 주식 종목의 주가 데이터를 분석합니다. 종목명이 틀릴 경우 유사한 종목을 제시합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_name": {
                    "type": "string",
                    "description": "분석할 종목명 (예: 삼성전자, LG화학)"
                },
                "start_date": {
                    "type": "string",
                    "description": "분석 시작일 (예: 2024-01-01)"
                },
                "end_date": {
                    "type": "string",
                    "description": "분석 종료일 (예: 2024-07-01)"
                }
            },
            "required": ["stock_name", "start_date", "end_date"]
        }
    }
]

# 🎯 사용자 입력
user_input = input("🔍 분석할 종목과 기간을 입력하세요 (예: 삼성전자 2024-01-01부터 2024-07-01까지): ")

# 🧠 GPT에 질문
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "당신은 똑똑한 한국 주식 분석 도우미입니다."},
        {"role": "user", "content": user_input}
    ],
    functions=functions,
    function_call="auto"
)

# 📥 GPT가 호출한 함수 파라미터 확인
function_call = response.choices[0].message.function_call
function_args = json.loads(function_call.arguments)

# 🔎 분석 실행
result = analyze_stock_price(**function_args)

# ⛔ 유효하지 않은 경우 바로 출력
if result.startswith("❌") or result.startswith("📭"):
    print(result)
else:
    # 📢 GPT에게 해석 요청
    final_response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": f"""
{result}
위 내용을 간단히 투자자 입장에서 해석해줘. 숫자는 그대로 써 주고, 길지 않게 써 줘.
"""}
        ],
        temperature=0.7
    )

    # 결과 출력
    print("\n✅ 주가 요약:\n")
    print(result)
    print("\n🤖 GPT 해석:\n")
    print(final_response.choices[0].message.content)


AttributeError: 'NoneType' object has no attribute 'arguments'